## This example shows implementation of Supervisor Agent Pattern, where all the request goes to one Supervisior Agent
### Supervisor agents is connected to all other agents 
### Supervisor agent determines to which agent to route the task


In [2]:
# Import the necessary libraries
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent


In [3]:
## Initialize the OpenAI model
from dotenv import load_dotenv
load_dotenv()
import os
open_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(temperature=0, api_key=open_api_key, model="gpt-4o-mini")

In [4]:
## Lets define the first tool
## Tools are the external functions that LLM can use to complete a task. 
## Remember, llm cannot provide latest information such as current weather, current stock price. Hence we need to create tools
## for realtime funciton

from langchain_community.tools import DuckDuckGoSearchRun

def search_duckduckgo(query:str):
    """Search DuckDuckGo usng Langchain's DuckDuckGo tool"""
    search = DuckDuckGoSearchRun()
    return search.run(query)

## Example run 
#result = search_duckduckgo("What are AI Agent")
#print(result)


AI agents are also difficult to evaluate in a repeatable way that shows progress without employing artificial constraints. This is especially challenging as the core capabilities of the underlying LLMs continue to rapidly improve, which makes it difficult to know whether your approach has improved results or if it's simply the underlying ... An AI agent is a software component that has the agency to act on behalf of a user or a system to perform tasks.Users can organize agents into systems that can orchestrate complex workflows, coordinate activities among multiple agents, apply logic to thorny problems, and evaluate answers to user queries. Limitations of AI Agents. AI agents have undoubtedly transformed many industries, but they come with limitations that impact their reliability, adaptability, and ethical use. Here are some of the main limitations: Limited Understanding of Context Many AI agents lack the deep understanding of context that humans have. An AI agent is a software progr

In [5]:
## Lets define additional tools

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

In [6]:
## Lets create Agents that perform specfic actions
math_agent = create_react_agent(
model = model,
tools = [add, multiply],
name="math_expert",
prompt="You are a math expert, Always use one tool at a time")

In [7]:
## Lets create research agent
research_agent  = create_react_agent(
model=model,
tools=[search_duckduckgo],
name="research_agent",
prompt="You are a world class researcher with access to websearch. Do not do any math ")

In [8]:
## Lets create a SUpervisor agent that will supervise the above agents
workflow = create_supervisor(
[research_agent,math_agent],
model = model,
prompt=(
    "You are a team supervisor managing a research expert and math expert. "
    "For current events use research_agent"
    "For math problems use math_agent"
)
)

In [9]:
## Fist step in Langgraph is to compile the workflow before using it
app = workflow.compile()

In [10]:
result = app.invoke({
    "messages":[
        {
            "role": "user",
            "content": "What is a quantum computing"
        }
    ]
})

In [11]:
print(result)

{'messages': [HumanMessage(content='What is a quantum computing', additional_kwargs={}, response_metadata={}, id='5e8b0d17-4a1f-41a7-8f01-1aa3614082c1'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3bDwJex3VWMj7B07PBE0cNh3', 'function': {'arguments': '{}', 'name': 'transfer_to_research_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 95, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLczIfbT4cezlu7Mv6lHBjMwaptyk', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-a69af6f9-a924-42a8-b0ad-08ddc2ee6fbf-0', tool_calls=[{'name': 'transfer_to_research_agent', 'args': {}, 'id': 'call_3bDwJex3VWM

In [12]:
## to see the results in a readable format
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is a quantum computing
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_agent (call_3bDwJex3VWMj7B07PBE0cNh3)
 Call ID: call_3bDwJex3VWMj7B07PBE0cNh3
  Args:
================================= Tool Message =================================
Name: transfer_to_research_agent

Successfully transferred to research_agent
================================== Ai Message ==================================
Name: research_agent

Quantum computing is a type of computing that utilizes the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the smallest unit of data (representing either a 0 or a 1), quantum computers use quantum bits, or qubits. Qubits can exist in multiple states simultaneously due to a property known as superposition, allowing them to represent both 0 and 1 at the sa

In [13]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is the weather in charlotte NC today. Multiply it by 2 and add 5"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is the weather in charlotte NC today. Multiply it by 2 and add 5
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_agent (call_i96jOwX81zUWaiElv7Yyiqb8)
 Call ID: call_i96jOwX81zUWaiElv7Yyiqb8
  Args:
================================= Tool Message =================================
Name: transfer_to_research_agent

Successfully transferred to research_agent
================================== Ai Message ==================================
Name: research_agent

Today in Charlotte, NC, the weather is expected to be partly cloudy with temperatures ranging from a maximum of 66.2°F to a minimum of 55.4°F. There is also a chance of light rain later in the day. 

If you need any further information or details, feel free to ask!
================================== Ai Message ==================================
Name: research_agent
